In [6]:
import json

# Load the JSON data from data.json
with open('data/custom.json') as f:
    data = json.load(f)



In [7]:
food_names = list(map(lambda f: f['name'], data['foods']))
food_names

['Buckwheat groats, dry, roasted',
 'Chicken, raw, meat only, boneless, skinless, breast, broiler or fryers',
 'Olive oil',
 'Fish, raw, wild, Atlantic, salmon',
 'Yogurt',
 'Orange',
 'Brazil nut',
 'Carrots, raw',
 'Oil, wheat germ',
 'Kale, raw']

In [8]:
nutrients = {}
nutrients_types = set()
for food in data['foods']:
    print(food)
    food_name = food['name']
    nutrients[food_name] = {}
    for nutrient in food['nutrients']:
        nutrients[food_name][nutrient] = food['nutrients'][nutrient]['value']
        nutrients_types.add(nutrient)

nutrients

{'name': 'Buckwheat groats, dry, roasted', 'nutrients': {'calories': {'value': 350, 'sources': ['Manufacturer']}, 'carbohydrates': {'value': 65, 'sources': ['Manufacturer']}, 'protein': {'value': 13, 'sources': ['Manufacturer']}, 'fat': {'value': 2.6, 'sources': ['Manufacturer']}, 'vitamin_d': {'value': 0, 'sources': ['https://www.nutritionvalue.org/Buckwheat_groats%2C_dry%2C_roasted_nutritional_value.html?size=100+g']}, 'iron': {'value': 2.47, 'measure': 'mg', 'sources': ['https://www.nutritionvalue.org/Buckwheat_groats%2C_dry%2C_roasted_nutritional_value.html?size=100+g']}, 'calcium': {'value': 17, 'measure': 'mg', 'sources': ['https://www.nutritionvalue.org/Buckwheat_groats%2C_dry%2C_roasted_nutritional_value.html?size=100+g']}, 'vitamin_c': {'value': 0, 'measure': 'mg', 'sources': ['https://www.nutritionvalue.org/Buckwheat_groats%2C_dry%2C_roasted_nutritional_value.html?size=100+g']}, 'saturated_fat': {'value': 0.6, 'measure': 'g', 'sources': ['Manufacturer']}, 'vitamin_a': {'value

{'Buckwheat groats, dry, roasted': {'calories': 350,
  'carbohydrates': 65,
  'protein': 13,
  'fat': 2.6,
  'vitamin_d': 0,
  'iron': 2.47,
  'calcium': 17,
  'vitamin_c': 0,
  'saturated_fat': 0.6,
  'vitamin_a': 0,
  'fiber': 10.3,
  'vitamin_e': 0,
  'vitamin_k': 1.9,
  'selenium': 8.4},
 'Chicken, raw, meat only, boneless, skinless, breast, broiler or fryers': {'calories': 120,
  'protein': 23,
  'carbohydrates': 0,
  'fat': 2.6,
  'vitamin_d': 0,
  'iron': 0.37,
  'calcium': 5,
  'vitamin_c': 0,
  'saturated_fat': 0.6,
  'vitamin_a': 9,
  'fiber': 0,
  'vitamin_e': 0.56,
  'vitamin_k': 8.4,
  'selenium': 22.8},
 'Olive oil': {'calories': 884,
  'protein': 0,
  'carbohydrates': 0,
  'fat': 100,
  'vitamin_d': 0,
  'iron': 0.6,
  'calcium': 1,
  'vitamin_c': 0,
  'saturated_fat': 14,
  'vitamin_a': 0,
  'fiber': 0,
  'vitamin_e': 14.35,
  'vitamin_k': 60.2,
  'selenium': 0},
 'Fish, raw, wild, Atlantic, salmon': {'calories': 142,
  'protein': 20,
  'carbohydrates': 0,
  'fat': 6.3,

In [9]:
# nutrients['Cooked buckwheat groats'].get('calories', 0)

In [12]:
import pulp as pl
import copy
def find_infeasible_constraints(prob, constraints):
    for name, constraint in constraints.items():
        test_prob = copy.deepcopy(prob)
        test_prob.constraints.pop(name)
        test_prob.solve()
        if pl.LpStatus[test_prob.status] != 'Infeasible':
            print(f"Constraint {name} is likely causing infeasibility")
        # else:
        #     print(f"Constraint {name} is not causing infeasibility")

prob = pl.LpProblem("Meal_Planning_Problem", pl.LpMinimize)
food_vars = pl.LpVariable.dicts("Amount", food_names, lowBound=0)
prob += pl.lpSum([food_vars[f] for f in food_names]), "Total Weight of Food"

calories = 2500
prob += pl.lpSum([(nutrients[f].get('calories', 0) * food_vars[f]) for f in food_names] 
                    ) >= calories, "Min Calories"

def add_nutrient_constraints(prob, nutrients, food_vars, nutrient_name, min_value, max_value):
    if nutrient_name not in nutrients_types:
        raise ValueError(f"Nutrient {nutrient_name} not found in the data")
    # Adding constraints for minimum nutrient values, if specified
    if min_value is not None:
        prob += pl.lpSum([(nutrients[f].get(nutrient_name, 0) * food_vars[f]) for f in food_names] 
                         ) >= min_value, f"Min {nutrient_name}"
    if max_value is not None:
        prob += pl.lpSum([(nutrients[f].get(nutrient_name, 0) * food_vars[f]) for f in food_names] 
                         ) <= max_value, f"Max {nutrient_name}"

add_nutrient_constraints(prob, nutrients, food_vars, 'carbohydrates', calories * 0.45 / 4,  calories * 0.65 / 4)
add_nutrient_constraints(prob, nutrients, food_vars, 'protein', calories * 0.1 / 4,  calories * 0.35 / 4)
add_nutrient_constraints(prob, nutrients, food_vars, 'fat', calories * 0.0 / 9,  calories * 0.3 / 9)
add_nutrient_constraints(prob, nutrients, food_vars, 'saturated_fat', 0,  calories * 0.1 / 9)
add_nutrient_constraints(prob, nutrients, food_vars, 'vitamin_d', 15, 100) # mcg
add_nutrient_constraints(prob, nutrients, food_vars, 'iron', 8, 45) # mg
add_nutrient_constraints(prob, nutrients, food_vars, 'calcium', 1000, 2500) # mg
add_nutrient_constraints(prob, nutrients, food_vars, 'vitamin_c', 90, 2000) # mg
add_nutrient_constraints(prob, nutrients, food_vars, 'vitamin_a', 900, 3000) # mcg
add_nutrient_constraints(prob, nutrients, food_vars, 'fiber', 38, None) # g
add_nutrient_constraints(prob, nutrients, food_vars, 'vitamin_e', 15, 1000) # mg
add_nutrient_constraints(prob, nutrients, food_vars, 'vitamin_k', 120, None) # mcg
add_nutrient_constraints(prob, nutrients, food_vars, 'selenium', 55, 400) # mcg

prob.solve()
print("Status:", pl.LpStatus[prob.status])
for v in prob.variables():
    if v.varValue > 0:
        formatted_name = v.name.replace("Amount_", "").replace("_", " ")
        print(f"{formatted_name} = {v.varValue * 100} grams")
# Print nutrients for the optimal solution

for nutrient in nutrients_types:
    nutrient_value = pl.lpSum([(nutrients[f].get(nutrient, 0) * food_vars[f].varValue) for f in food_names])
    print(f"{nutrient}: {nutrient_value}")


# Collect all constraints
constraints = {name: prob.constraints[name] for name in prob.constraints.keys()}

# Find infeasible constraints
find_infeasible_constraints(prob, constraints)

Status: Optimal
Brazil nut = 16.020514 grams
Buckwheat groats, dry, roasted = 476.29839 grams
Carrots, raw = 6.4581563 grams
Fish, raw, wild, Atlantic, salmon = 136.36364 grams
Kale, raw = 344.27832 grams
Oil, wheat germ = 2.9794945999999998 grams
Olive oil = 43.468336 grams
vitamin_c: 322.0491256997
fat: 83.3333335526
saturated_fat: 14.051262013616
vitamin_k: 1378.5234451198003
vitamin_a: 899.999993105
selenium: 400.00000977630003
vitamin_d: 15.000000400000001
protein: 101.4765853467
fiber: 64.55651221640001
vitamin_e: 15.000000052979999
carbohydrates: 327.5706757464
iron: 19.0286092939
calories: 2499.9999996470005
calcium: 999.999993239
Constraint Min_Calories is likely causing infeasibility
Constraint Min_carbohydrates is likely causing infeasibility
Constraint Max_carbohydrates is likely causing infeasibility
Constraint Min_protein is likely causing infeasibility
Constraint Max_protein is likely causing infeasibility
Constraint Min_fat is likely causing infeasibility
Constraint Max

Included Nutrients

Vitamin D
Iron
Calcium
Vitamin C
Vitamin A
Fiber
Vitamin E
Vitamin K
Selenium






B Vitamins:
B1 (Thiamine): 1.1-1.2 mg/day
B2 (Riboflavin): 1.1-1.3 mg/day
B3 (Niacin): 14-16 mg/day
B5 (Pantothenic Acid): 5 mg/day
B6 (Pyridoxine): 1.3-1.7 mg/day
B7 (Biotin): 30 µg/day
B9 (Folate): 400 µg/day
B12 (Cobalamin): 2.4 µg/day
Minerals:

Calcium: 1,000-1,300 mg/day
Iron: 8-18 mg/day
Magnesium: 310-420 mg/day
Phosphorus: 700 mg/day
Potassium: 2,500-3,400 mg/day
Sodium: Less than 2,300 mg/day
Zinc: 8-11 mg/day
Iodine: 150 µg/day

Copper: 900 µg/day
Manganese: 1.8-2.3 mg/day
Fluoride: 3-4 mg/day
Antioxidants: Various amounts depending on the food source (e.g., flavonoids, polyphenols)

Phytochemicals: No specific RDI, but beneficial for health (e.g., carotenoids, flavonoids)

Probiotics: No specific RDI, but beneficial for gut health (e.g., from yogurt, kefir)